In [1]:
import gpxpy
import numpy as np
import pandas as pd
from kompy import KomootConnector
import os
connector = KomootConnector(
    email=os.environ['KOMOOT_EMAIL'],
    password=os.environ['KOMOOT_PASSWORD'],
)
gpx_track = connector.get_tour_by_id(tour_identifier='1378844177', object_type='gpx')


In [2]:
gpx_track

GPX(tracks=[GPXTrack(name='W02D2-Recovery Run', segments=[GPXTrackSegment(points=[...])])])

In [3]:
from gpxpy.gpx import GPXTrackPoint
import math
from geopy import distance

def calculate_initial_compass_bearing(first_point: GPXTrackPoint, second_point: GPXTrackPoint):
    """
    Calculates the bearing between two points.

    The formula used to calculate the bearing is:
        θ = atan2(sin(Δlong).cos(lat2), cos(lat1).sin(lat2) - sin(lat1).cos(lat2).cos(Δlong))

    :param first_point: The first point
    :param second_point: The second point
    :return: The bearing between the two points in degrees
    """

    lat1 = math.radians(first_point.latitude)
    lat2 = math.radians(second_point.latitude)
    diffLong = math.radians(second_point.longitude - first_point.longitude)

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Normalize the initial bearing
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

def calculate_turn_angle(bearing1, bearing2):
    """
    Calculate the angle of turn between two bearings.
    """
    angle = bearing2 - bearing1
    
    angle = (angle + 180) % 360 - 180

    return angle




def load_gpx_todf(gpx_data):
    gpx_points = []
    for track in gpx_data.tracks:
        for segment in track.segments:
            for point in segment.points:
                gpx_points.append(
                    {
                        'gpx_point': point,
                        "latitude": point.latitude,
                        "longitude": point.longitude,
                        "elevation": point.elevation,
                        "time": point.time,
                        "distance": distance.geodesic(
                            (point.latitude, point.longitude),
                            (gpx_points[-1]['latitude'],gpx_points[-1]['longitude']),
                        ).m if len(gpx_points) > 0 else 0,
                        'time_diff_s': (point.time - gpx_points[-1]['gpx_point'].time).total_seconds() if len(gpx_points) > 0 else 0,
                        'elevation_diff': point.elevation - gpx_points[-1]['gpx_point'].elevation if len(gpx_points) > 0 else 0,
                        'bearing': calculate_initial_compass_bearing(
                            first_point=point,
                            second_point=gpx_points[-1]['gpx_point'],
                        ) if len(gpx_points) > 0 else 0,
                    }
                )
    return pd.DataFrame(gpx_points)

In [4]:
gpx_df = load_gpx_todf(gpx_track)

In [19]:
WINDOW_SIZE = 6

In [20]:
gpx_df['turn_angle'] = gpx_df['bearing'].rolling(window=WINDOW_SIZE,min_periods=1).apply(
    lambda x: calculate_turn_angle(
        bearing1=np.average(x[:WINDOW_SIZE//2]),
        bearing2=np.average(x[WINDOW_SIZE//2:]),
        # bearing1=GPXTrackPoint(latitude=np.average([point.latitude for point in x[:WINDOW_SIZE//2]]), longitude=np.average([point.longitude for point in x])),
        # bearing2=GPXTrackPoint(latitude=np.average([point.latitude for point in x[WINDOW_SIZE//2:]]), longitude=np.average([point.longitude for point in x])),
    )
)

/Users/matteo.villosio/PycharmProjects/describe-trail/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning:

Mean of empty slice.



In [21]:
gpx_df

,gpx_point,latitude,longitude,elevation,time,distance,time_diff_s,elevation_diff,bearing,turn_angle,smoothed_turn_angle,turn_extrema,elevation_extrema,turn_direction
0,"[trkpt:45.736284,7.315084@550.838341@2023-11-2...",45.736284,7.315084,550.838341,2023-11-20 17:02:16+00:00,0.000000,0.0,0.0,0.000000,NaN,NaN,0.000000,550.838341,0.0
1,"[trkpt:45.73628,7.315069@550.838341@2023-11-20...",45.736280,7.315069,550.838341,2023-11-20 17:02:17+00:00,1.249245,1.0,0.0,69.089899,NaN,NaN,0.000000,550.838341,0.0
2,"[trkpt:45.736266,7.315043@550.838341@2023-11-2...",45.736266,7.315043,550.838341,2023-11-20 17:02:18+00:00,2.552687,1.0,0.0,52.350637,NaN,NaN,0.000000,550.838341,0.0
3,"[trkpt:45.73625,7.315014@550.838341@2023-11-20...",45.736250,7.315014,550.838341,2023-11-20 17:02:19+00:00,2.873486,1.0,0.0,51.674378,11.194199,NaN,0.000000,550.838341,0.0
4,"[trkpt:45.736222,7.31498@550.838341@2023-11-20...",45.736222,7.314980,550.838341,2023-11-20 17:02:20+00:00,4.085058,1.0,0.0,40.282128,5.498074,NaN,0.000000,550.838341,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3704,"[trkpt:45.735716,7.315114@551.559318@2023-11-2...",45.735716,7.315114,551.559318,2023-11-20 18:04:00+00:00,1.336023,1.0,0.0,183.328807,-29.888061,1.369525,0.000000,551.559318,0.0
3705,"[trkpt:45.735729,7.315104@551.559318@2023-11-2...",45.735729,7.315104,551.559318,2023-11-20 18:04:01+00:00,1.641191,1.0,0.0,151.768682,-36.297006,-6.872323,0.000000,551.559318,0.0
3706,"[trkpt:45.735752,7.315098@551.559318@2023-11-2...",45.735752,7.315098,551.559318,2023-11-20 18:04:02+00:00,2.598665,1.0,0.0,169.680698,-40.862475,-14.832217,0.000000,551.559318,0.0
3707,"[trkpt:45.735774,7.315096@551.559318@2023-11-2...",45.735774,7.315096,551.559318,2023-11-20 18:04:03+00:00,2.450165,1.0,0.0,176.369353,-29.684506,-21.482519,0.000000,551.559318,0.0


In [22]:
from scipy import signal
import numpy as np

gpx_df['smoothed_turn_angle'] = gpx_df['turn_angle'].rolling(window=10,min_periods=1).mean()



In [23]:
from scipy.signal import argrelextrema


def identify_local_minmax(df, input_column_name, output_column_name='relative_extrema', order=None):
    # Initialize a new column 'turn' with NaN values
    df[output_column_name] = 0

    # Identify indices of local minima and maxima
    if not order:
        minima_indices = argrelextrema(df[input_column_name].values, np.less_equal)[0]
        maxima_indices = argrelextrema(df[input_column_name].values, np.greater_equal)[0]
    else:
        minima_indices = argrelextrema(df[input_column_name].values, np.less_equal, order=order)[0]
        maxima_indices = argrelextrema(df[input_column_name].values, np.greater_equal, order=order)[0]

    # Mark local minima and maxima in the 'turn' column
    df.loc[minima_indices, output_column_name] = df.loc[minima_indices, input_column_name]
    df.loc[maxima_indices, output_column_name] = df.loc[maxima_indices, input_column_name]

    return df

In [24]:
gpx_df = identify_local_minmax(gpx_df, input_column_name='smoothed_turn_angle', output_column_name='turn_extrema', 
                               order=70)
gpx_df = identify_local_minmax(gpx_df, input_column_name='elevation', output_column_name='elevation_extrema')
gpx_df['turn_direction'] = np.sign(gpx_df['turn_extrema'])

/var/folders/ks/7zd0y9gn3b1f5y7zw35zrm9r0000gq/T/ipykernel_62567/1385703926.py:17: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-56.13516607 -36.57624894 -40.18256309 -11.5048083  -16.31834771
 -22.71590872 -32.75596795 -23.77661906 -19.6891467  -18.84516316
  -5.65219743  -2.91452704  -2.68410975  -4.13906219  -3.99549038
 -31.96871497  -4.57590087 -19.27797827  -2.31246683  -4.21453821
 -43.86883566 -21.40617956 -22.49974013 -22.84811398 -22.79135106]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/ks/7zd0y9gn3b1f5y7zw35zrm9r0000gq/T/ipykernel_62567/1385703926.py:17: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[550.838341 550.838341 550.838341 550.838341 550.838341 550.838341
 550.838341 550.838341 550.838341 550.838341 550.838341 550.838341
 550.838341 550.838341 550.838341 550.8383

In [25]:
gpx_df

,gpx_point,latitude,longitude,elevation,time,distance,time_diff_s,elevation_diff,bearing,turn_angle,smoothed_turn_angle,turn_extrema,elevation_extrema,turn_direction
0,"[trkpt:45.736284,7.315084@550.838341@2023-11-2...",45.736284,7.315084,550.838341,2023-11-20 17:02:16+00:00,0.000000,0.0,0.0,0.000000,NaN,NaN,0.000000,550.838341,0.0
1,"[trkpt:45.73628,7.315069@550.838341@2023-11-20...",45.736280,7.315069,550.838341,2023-11-20 17:02:17+00:00,1.249245,1.0,0.0,69.089899,NaN,NaN,0.000000,550.838341,0.0
2,"[trkpt:45.736266,7.315043@550.838341@2023-11-2...",45.736266,7.315043,550.838341,2023-11-20 17:02:18+00:00,2.552687,1.0,0.0,52.350637,NaN,NaN,0.000000,550.838341,0.0
3,"[trkpt:45.73625,7.315014@550.838341@2023-11-20...",45.736250,7.315014,550.838341,2023-11-20 17:02:19+00:00,2.873486,1.0,0.0,51.674378,11.194199,11.194199,0.000000,550.838341,0.0
4,"[trkpt:45.736222,7.31498@550.838341@2023-11-20...",45.736222,7.314980,550.838341,2023-11-20 17:02:20+00:00,4.085058,1.0,0.0,40.282128,5.498074,8.346136,0.000000,550.838341,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3704,"[trkpt:45.735716,7.315114@551.559318@2023-11-2...",45.735716,7.315114,551.559318,2023-11-20 18:04:00+00:00,1.336023,1.0,0.0,183.328807,-29.888061,-8.730693,0.000000,551.559318,0.0
3705,"[trkpt:45.735729,7.315104@551.559318@2023-11-2...",45.735729,7.315104,551.559318,2023-11-20 18:04:01+00:00,1.641191,1.0,0.0,151.768682,-36.297006,-13.307960,0.000000,551.559318,0.0
3706,"[trkpt:45.735752,7.315098@551.559318@2023-11-2...",45.735752,7.315098,551.559318,2023-11-20 18:04:02+00:00,2.598665,1.0,0.0,169.680698,-40.862475,-18.021087,0.000000,551.559318,0.0
3707,"[trkpt:45.735774,7.315096@551.559318@2023-11-2...",45.735774,7.315096,551.559318,2023-11-20 18:04:03+00:00,2.450165,1.0,0.0,176.369353,-29.684506,-21.448561,0.000000,551.559318,0.0


In [26]:
import plotly.express as px


In [27]:
import plotly.graph_objects as go

scatter = go.Scatter(
    x=gpx_df['time'],
    y=gpx_df['elevation'],
    mode='lines',
)

trace2 = go.Scatter(
    x=gpx_df[gpx_df['elevation_extrema'] != 0]['time'],
    y=gpx_df[gpx_df['elevation_extrema'] != 0]['elevation_extrema'],
    mode='markers',
)

fig = go.Figure(data=[scatter, trace2])
fig.show()

In [28]:
scatter = go.Scatter(
    x=gpx_df['time'],
    y=gpx_df['smoothed_turn_angle'],
    mode='lines',
)

trace2 = go.Scatter(
    x=gpx_df[gpx_df['turn_extrema'] != 0]['time'],
    y=gpx_df[gpx_df['turn_extrema'] != 0]['turn_extrema'],
    mode='markers',
)

fig = go.Figure(data=[scatter, trace2])
fig.show()


In [29]:

fig = px.scatter_mapbox(
    gpx_df[:3000],
    lat="latitude",
    lon="longitude",
    hover_name="time_diff_s",
    hover_data=['smoothed_turn_angle', "turn_angle"],
    color="turn_direction",
    zoom=18,
    width=1200,
    height=700,
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()